In [6]:
import lark
from lark import Lark, Transformer, v_args

In [433]:
kumir_robot = """
    start: "использовать" "Робот"i algs

    algs: alg+

    alg: "алг" name body 

    name: CHAR | CHAR name

    commands: command+
    
    body: "нач" commands "кон"

    command: cycle | (action";")* action | if_expression 

    action: /вверх/i | /вниз/i | /влево/i | /вправо/i | /закрасить/i

    cycle: "нц" (NUM /раз/ | while_cycle) command+ ("кц" | "кц_при" condition)

    while_cycle: "пока" condition

    condition: NOT? direction wall

    wall: /свободно/i | /стена/i

    direction: /слева/i | /справа/i | /сверху/i | /снизу/i

    if_expression: "если" condition "то" command+ (else_expression | "все")

    else_expression: "иначе" command+ "все"

    NOT: "не"

    CHAR: /[|а-яА-Я0-9" "]+/

    VAR: /[a-zA-Z]/

    NUM: /[0-9]+/

%import common.NEWLINE
%import common.WS_INLINE
%ignore WS_INLINE
%ignore NEWLINE
"""

In [476]:
class RobotInterpreter(Transformer):

    def start(self, items):
        return ''

    def alg(self, items):
        name = items[0]
        body = items[1]
        print(f"Парсинг алгоритма{name} завершен успешно.")
        return body

    def name(self, items):
        return ''.join(str(item) for item in items)

    def action(self, token):
        act = str(token[0]).lower()
        print(f"Обнаружено действие: {act}")

    def cycle(self, items):
        cycle_type = items[0]
        commands = items[1]
        if cycle_type[0] == 'while':
            print(f"Обнаружен цикл типа пока")
            return ('while', cycle_type[1], commands)
        else:
            count = cycle_type
            print(f"Обнаружен цикл типа повторение {count} раз")
            return ('repeat', count, commands)

    def while_cycle(self, items):
        condition = items[0]
        return ('while', condition)

    def condition(self, items):
        if len(items)==2:
            negation = False
            direction = items[0]
            wall = items[1]
        else:
            negation = True
            direction = items[1]
            wall = items[2]
        if negation:
            print(f"Обнаружено условие: {direction} не {wall}")
        else:
            print(f"Обнаружено условие: {direction} {wall}")

    def wall(self, token):
        return str(token[0]).lower()

    def direction(self, token):
        return str(token[0]).lower()

    def NOT(self, token):
        return str(token)

пробую вложенность

In [540]:
class RobotInterpreter(Transformer):
    def start(self, items):
        # Здесь items — список, содержащий корень или главный блок
        # Обрабатываем его рекурсивно
        # Например, если это список блоков
        result = ''
        for item in items:
            if isinstance(item, Tree):
                result += self._process_node(item)
            else:
                result += str(item)
        return result

    def alg(self, items):
        name = items[0]
        body = items[1]
        print(f"Парсинг алгоритма {name} завершен успешно.")
        return body

    def name(self, items):
        # Собираем имя алгоритма
        return ''.join(str(item) for item in items)

    def commands(self, items):
        print("SAHDLHD", items)

    def command(self, items):
        print("AAAA", items)

    def action(self, token):
        act = str(token[0]).lower()
        print(f"Обнаружено действие: {act}")
        return act

    def cycle(self, items):
        cycle_type = items[0]
        commands = items[1]
        if isinstance(cycle_type, Tree) and cycle_type.data == 'while_cycle':
            print(f"Обнаружен цикл типа пока")
            return ('while', cycle_type.children[0], commands)
        else:
            count = int(cycle_type)
            print(f"Обнаружен цикл типа повторение {count} раз")
            return ('repeat', count, commands)

    def while_cycle(self, items):
        condition = items[0]
        return Tree('while_cycle', [condition])

    def condition(self, items):
        if len(items) == 2:
            negation = False
            direction = items[0]
            wall = items[1]
        else:
            negation = True
            direction = items[1]
            wall = items[2]
        if negation:
            print(f"Обнаружено условие: {direction} не {wall}")
        else:
            print(f"Обнаружено условие: {direction} {wall}")
        return {
            'negation': negation,
            'direction': direction,
            'wall': wall
        }

    def wall(self, token):
        return str(token[0]).lower()

    def direction(self, token):
        return str(token[0]).lower()

    def NOT(self, token):
        return str(token)

    def _process_node(self, node):
        # Вспомогательный метод для рекурсивной обработки узлов
        if isinstance(node, Tree):
            method = getattr(self, node.data, None)
            if method:
                return method(node.children)
            else:
                # Если метода нет, обрабатываем дочерние узлы
                result = ''
                for child in node.children:
                    if isinstance(child, Tree):
                        result += self._process_node(child)
                    else:
                        result += str(child)
                return result
        elif isinstance(node, Token):
            return str(node)
        else:
            return str(node)

In [541]:
parser = Lark(kumir_robot, start='start', parser='lalr')

In [542]:
input_string = """
использовать Робот 
алг ура
нач
нц пока сверху свободно 
  если не справа свободно то
    закрасить
  все
кц
закрасить
нц 3 раз
влево
кц
кон
"""

In [543]:
try:
    parsed_tree = parser.parse(input_string)
    print("Parsed Tree:", parsed_tree.pretty())
except Exception as e:
    print(f"Ошибка при разборе ввода: {e}")

Parsed Tree: start
  algs
    alg
      name	 ура
      body
        commands
          command
            cycle
              while_cycle
                condition
                  direction	сверху
                  wall	свободно
              command
                if_expression
                  condition
                    не
                    direction	справа
                    wall	свободно
                  command
                    action	закрасить
          command
            action	закрасить
          command
            cycle
              3
              раз
              command
                action	влево



In [544]:
from pprint import pprint
try:
    transformed_result = RobotInterpreter().transform(parsed_tree)
    pprint(transformed_result)
except Exception as e:
    print(f"Ошибка при разборе ввода: {e}")

Обнаружено условие: сверху свободно
Обнаружено условие: справа не свободно
Обнаружено действие: закрасить
AAAA ['закрасить']
AAAA [Tree(Token('RULE', 'if_expression'), [{'negation': True, 'direction': 'справа', 'wall': 'свободно'}, None])]
Обнаружен цикл типа пока
AAAA [('while', {'negation': False, 'direction': 'сверху', 'wall': 'свободно'}, None)]
Обнаружено действие: закрасить
AAAA ['закрасить']
Обнаружено действие: влево
AAAA ['влево']
Обнаружен цикл типа повторение 3 раз
AAAA [('repeat', 3, Token('__ANON_5', 'раз'))]
SAHDLHD [None, None, None]
Парсинг алгоритма  ура завершен успешно.
'None'


In [504]:
input_string = """
использовать Робот
алг из А в Б
нач
    вправо
    нц 5 раз
        вверх; вверх; вправо; вниз; вниз; вправо
    кц
кон
"""

In [369]:
try:
    parsed_tree = parser.parse(input_string)
    print("Parsed Tree:", parsed_tree.pretty())
except Exception as e:
    print(f"Ошибка при разборе ввода: {e}")


Parsed Tree: start
  alg
    name	 из А в Б
    body
      command
        action	вправо
      command
        cycle
          5
          раз
          command
            action	вверх
            action	вверх
            action	вправо
            action	вниз
            action	вниз
            action	вправо



In [370]:
try:
    transformed_result = RobotInterpreter().transform(parsed_tree)
    print(transformed_result)
except Exception as e:
    print(f"Ошибка при разборе ввода: {e}")

Обнаружено действие: вправо
Обнаружено действие: вверх
Обнаружено действие: вверх
Обнаружено действие: вправо
Обнаружено действие: вниз
Обнаружено действие: вниз
Обнаружено действие: вправо
Обнаружен цикл типа повторение 5 раз
Парсинг алгоритма из А в Б завершен успешно.



In [375]:
input_string = """
использовать Робот
алг ААА
нач
    вправо
    нц 5 раз
        если сверху свободно то
            вверх; вверх; вправо; вниз; вниз; вправо
        иначе
            закрасить
            вправо
        все
    кц
кон
"""

In [376]:
try:
    parsed_tree = parser.parse(input_string)
    print("Parsed Tree:", parsed_tree.pretty())
except Exception as e:
    print(f"Ошибка при разборе ввода: {e}")


Parsed Tree: start
  alg
    name	 ААА
    body
      command
        action	вправо
      command
        cycle
          5
          раз
          command
            if_expression
              condition
                direction	сверху
                wall	свободно
              command
                action	вверх
                action	вверх
                action	вправо
                action	вниз
                action	вниз
                action	вправо
              else_expression
                command
                  action	закрасить
                command
                  action	вправо



In [377]:
try:
    transformed_result = RobotInterpreter().transform(parsed_tree)
    print(transformed_result)
except Exception as e:
    print(f"Ошибка при разборе ввода: {e}")

Обнаружено действие: вправо
Обнаружено условие: сверху свободно
Обнаружено действие: вверх
Обнаружено действие: вверх
Обнаружено действие: вправо
Обнаружено действие: вниз
Обнаружено действие: вниз
Обнаружено действие: вправо
Обнаружено действие: закрасить
Обнаружено действие: вправо
Обнаружен цикл типа повторение 5 раз
Парсинг алгоритма ААА завершен успешно.

